In [2]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import collections
import asyncio
import getpass
import re
from datetime import datetime as dt
import os,sys,re
import urllib3
import prettytable
from collections import Counter
import seaborn as sns
import random
from datetime import timedelta

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
#code used to ping the postgres mimic server. 

conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

time: 2.55 s


In [6]:
ABrx = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/02082018_ABrx_updated.csv')


time: 859 ms


In [11]:
ABrx.head()

,Unnamed: 0,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,...,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route,rxcui,Antibiotics
0,16,2366833,24915,194424,267059.0,2182-12-10,2182-12-10,MAIN,Daptomycin,NaN,...,53190.0,6.791900e+10,500mg Vial,350,mg,0.7,VIAL,IV,404652.0,True
1,28,1867722,24922,165593,268898.0,2165-01-02,2165-01-08,MAIN,Ceftazidime,NaN,...,15538.0,1.730412e+08,1GM FROZ. BAG,1,gm,1,BAG,IV,NaN,False
2,75,2804000,24942,178112,278729.0,2159-11-08,2159-11-10,MAIN,Vancomycin HCl,NaN,...,43952.0,3.383552e+08,1g Frozen Bag,1000,mg,1,BAG,IV,1807508.0,True
3,98,993369,24955,154989,NaN,2142-09-21,2142-10-08,MAIN,Sulfameth/Trimethoprim SS,Sulfameth/Trimethoprim SS,...,9395.0,9.300880e+07,1 TAB,1,TAB,1,TAB,PO,198334.0,True
4,117,2338288,24968,127995,NaN,2190-07-14,2190-07-15,MAIN,NEO*IV*Gentamicin,NaN,...,9298.0,6.410394e+08,10mg/mL-2mL,5,mg,0.5,ml,VT,NaN,False


time: 24.7 ms


In [24]:
ABrx_Drug = pd.DataFrame(ABrx.loc[ABrx['Antibiotics'] == True, 'drug'].unique())

ABrx_Drug_list = list(ABrx_Drug[0])
len(ABrx_Drug_list)#202

time: 14.2 ms


# Generating antibiotic list from inputevents_mv table

## making a clean list of AB labels from ATC and D_items

note the code in this section was generated for the purpose of comparing antibiotic event lists from inputevents vs prescriptions. the prescription table was chosen as the best source for this (see below 
Problem:

    MIMICIII doesn't have a comprehensive list of antibiotics annotated.


solution strategy:

1)make a list of antibiotic names from two different sources and combine them:

    d_item list annotated with Antibiotic column
    
    prescription table annotated with ATC antibiotic info
    
2)use the list of antibiotics to find all the item id's in d_item list. 
    

In [7]:
#d_item_microbio_list generation
#the caviot list is due to antivirals and antibacterials being present in the antibiotic list
d_items_sql = query_schema + """
select itemid, label, dbsource, linksto, category, unitname, param_type, conceptid 
from mimiciii.d_items
WHERE (category = 'Antibiotics' or category = 'ANTIBACTERIUM') and lower(label) not like '%fung%' 
and lower(label) not in ('voriconazole','valgancyclovir','chloroquine','tamiflu','mefloquine','foscarnet','fluconazole','vorconazole','quinine',
'ribavirin','gancyclovir','chloroquine','atovaquone','ambisome', 'acyclovir', 'Acyclovir')
order by category, label desc;
"""

d_items_df=pd.read_sql_query(d_items_sql,conn)
d_item_microbio_list=d_items_df['label'].unique().tolist()
d_item_microbio_list=[str(x).lower() for x in d_item_microbio_list]



time: 289 ms


In [8]:
#d_item_microbio_list generation
#the caviot list is due to antivirals and antibacterials being present in the antibiotic list
d_items_sql = query_schema + """
select itemid, label, dbsource, linksto, category, unitname, param_type, conceptid 
from mimiciii.d_items
WHERE (category = 'Antibiotics' or category = 'ANTIBACTERIUM') and lower(label) not like '%fung%' 
and lower(label) not in ('voriconazole','valgancyclovir','chloroquine','tamiflu','mefloquine','foscarnet','fluconazole','vorconazole','quinine',
'ribavirin','gancyclovir','chloroquine','atovaquone','ambisome', 'acyclovir', 'Acyclovir')
order by category, label desc;
"""

d_items_df=pd.read_sql_query(d_items_sql,conn)
d_item_microbio_list=d_items_df['label'].unique().tolist()
d_item_microbio_list=[str(x).lower() for x in d_item_microbio_list]

time: 16.5 ms


In [9]:
#function to parse out ABrx_Drug_list
#uses regular esxpressions and rule based parsing

i=0
replace_list= [
    'sodium','hcl','potassium','sulfate','potassium','monohydrate', 
    'oral susp.','ophthalmic oint', 'desensitization', r'% solution',
    'hippurate','suspension','ointment','phosphate', 'soln', 'solution',
    'ophth.','ophth.','*nf*']
    
@memory.cache
def parsingfxn(element):
    element = r'%s'%(element)
    element = re.sub(r'\b\w{1,3}\b', r"", element)
    element= element.strip(r" ")
    for j in replace_list:
        element= element.replace(j, r'')   
    element= element.strip(r" ")
    if r"**" == str(element)[0:2]:
        element = str(element)[2:]
    element = element.split(r".%")[0]
    element = element.split(r". %")[0]
    element = element.split(r"ophth*")[0]
    element = element.split(r"-")[0]
    element = element.split(r"(")[0]
    element = element.split(' ',1)[0]
    return(element)    

time: 24.6 ms


In [27]:
ABrx_Drug_list3 =[parsingfxn(x) for x in ABrx_Drug_list]
ABrx_Drug_list3= list(set(ABrx_Drug_list3))
ABrx_Drug_list3.remove('')

time: 258 ms


In [28]:
#final consensus list coding
Combo_AB_name_list=[]
Combo_AB_name_list = ABrx_Drug_list3 + d_item_microbio_list
Combo_AB_name_list= list(set(Combo_AB_name_list))

#cleaning
#Combo_AB_name_list.remove('none')
Combo_AB_name_list.remove('poly')
Combo_AB_name_list.remove('eryth')
Combo_AB_name_list.remove('gent')
Combo_AB_name_list.remove('sulfa')
#Combo_AB_name_list.remove('genta')
Combo_AB_name_list.remove('bact')

print(len(Combo_AB_name_list))

142
time: 6.79 ms


In [29]:
kishore_list= ['amikacin',
 'amoxicillin','ampicillin','avelox','azithromycin','aztreonam','bacitracin','bactrim','benzathine',
 'cefazolin','cefepime','cefixime','cefotaxime','cefotetan', 'cefpodo',
 'cefpodoxime', 'ceftaroline',
 'ceftazidime','ceftriaxone','cephalexin','chloramphenicol','cilastatin',
 'ciprofloxacin','clarithromycin','clavulanic',
 'clindamycin','colistimethate','colistin','daptomycin',
 'demeclocycline','desensitization','dexamethasone','dicloxacillin',
 'doxycycline','ertapenem','erythromycin','ethylsuccinate','flagyl','fortified','fosfomycin','furosemide',
 'gentamic','gentamicin','gramicidin','hippurate',
 'imipenem','levofloxacin','linezolid','macrobid',
 'macrocrystal','macrodantin','meropenem','methenamine',
 'metronidazole','moxifloxacin','nafcillin','neomycin',
 'nffloxin','nitrofurantoin','ocuflox','ofloxacin','ophthalmic','oxacillin',
 'penicillin','piperacillin','polymyxin','polysporin',
 'proxetil','streptomycin','sulbactam',
 'sulfadiazine','sulfame','sulfamet','sulfameth','sulfamethoxazole',
 'tazobactam','tigecycline','tobramycin','trimeth',
 'trimetho','trimethopri','trimethoprim',
 'tromethamine','unasyn','vancomycin','vigamox','zithromax','zosyn']

time: 8.42 ms


In [30]:
consensus_kishore_mylist = []
consensus_kishore_mylist= kishore_list + Combo_AB_name_list
consensus_kishore_mylist= list(set(consensus_kishore_mylist))
len(consensus_kishore_mylist) 
#95->120 after removing antivirals/fungals on march 11. was at 135 before removing all erronious ab

194

time: 3.88 ms


In [31]:
#algorithm to loop through our consensus ab list of labels and 
#input them into a like command to find associated d_item id's

abid_test_list=[]
for element in consensus_kishore_mylist:

    d_items_sql2 = query_schema + """
    select itemid, label, dbsource, linksto, category, unitname, param_type, conceptid 
    from mimiciii.d_items
    WHERE lower(label) like '%{}%'
    order by itemid desc;
    """.format(element.strip())

    d_items_df2=pd.read_sql_query(d_items_sql2,conn)
    abid_test_list= abid_test_list +d_items_df2['itemid'].unique().tolist()
    abid_test_list= list(set(abid_test_list))

#using getting all d_item info from ab list    
d_items_sql3 = query_schema + """
    select itemid, label, dbsource, linksto, category, unitname, param_type, conceptid 
    from mimiciii.d_items
    WHERE itemid in {}
    order by itemid desc;
    """.format(tuple(abid_test_list))
    
d_items_df3=pd.read_sql_query(d_items_sql3,conn)    

len(abid_test_list) #163
print(len(d_items_df3['itemid'].unique()),len(abid_test_list) ) #both should be 163

163
time: 4.94 s


In [34]:
#QC
#making a list of item id's that were recruited in query, but are not antibiotics
#these will be removed from the antibiotic list going forward
@memory.cache
def d_item_list_generation_fxn(ablist_input):
    morphine_list= [1813,5613,7424,30153,30126,225154]
    lasix_list=[3439,228340,221794]
    magnesium_list = [6133, 30027, 222011, 227523, 227524]

    misc_list= [42565, 30024, 42041 ]
    #examples:
    #Tromethamine  - NOT AN AB -42565
    #ferrous sulfate - NOT AN AB-30024
    #barium sulfate- NOT AN AB- 42041

    all_rm_list= morphine_list + lasix_list + magnesium_list + misc_list
    
    EVENT_DITEM_ABLIST= list(ablist_input) 
    EVENT_DITEM_ABLIST = [x for x in EVENT_DITEM_ABLIST if x not in all_rm_list]
    return(EVENT_DITEM_ABLIST)

EVENT_DITEM_ABLIST= d_item_list_generation_fxn(d_items_df3['itemid'].unique())



time: 27.1 ms


# generating EVENT patient list
goal: now that we have ab list, need to generate list of all patients who have ever been prescribed ab

strategy: use series of sql like joins

In [ ]:
#on 4-17-18 made this outdated by generating a much more comprehensive patient info dataframe: ptinfo_ICUsterile_16_pt

#on 4/20 found a bug where pt 85 is not in ptinfo_ICUsterile_16. saving and regenerating
#update: was fixed after rerunnign the code...
#pd.DataFrame(ptinfo_ICUsterile_16).to_csv('ptinfo_ICUsterile_16_04-20-18.csv')

In [ ]:
#generating inputevents table 

###CV patients
####chart events
chartevents_sql = query_schema + """
select * from chartevents
WHERE itemid in {}""".format(tuple(FINAL_DITEM_ABLIST))

chartevents_df=pd.read_sql_query(chartevents_sql,conn)

####inputevents_cv
inputevents_cv_sql = query_schema + """
select * from inputevents_cv
WHERE itemid in {}""".format(tuple(FINAL_DITEM_ABLIST))

inputevents_cv_df=pd.read_sql_query(inputevents_cv_sql,conn)

####inputevents_mv
inputevents_mv_sql = query_schema + """
select * from mimiciii.inputevents_mv mv
left join (SELECT di.itemid, di.label from mimiciii.d_items di) di on di.itemid= mv.itemid
WHERE mv.itemid in {}""".format(tuple(FINAL_DITEM_ABLIST))


inputevents_mv_df=pd.read_sql_query(inputevents_mv_sql,conn) 

In [ ]:
inputevents_mv_sql = query_schema + """
select * from mimiciii.inputevents_mv mv
left join (SELECT di.itemid, di.label from mimiciii.d_items di) di on di.itemid= mv.itemid
WHERE mv.itemid in {}""".format(tuple(FINAL_DITEM_ABLIST))


inputevents_mv_df=pd.read_sql_query(inputevents_mv_sql,conn) 
inputevents_mv_df= inputevents_mv_df.ix[:,~inputevents_mv_df.columns.duplicated()] #removing the extra itemid